In [1]:
from Gaugi import load as gaugi_load
from kepler.pandas.readers import load as kepler_load

import pandas as pd
import numpy as np
import os

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence
INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
def load_as_df(filepath: str) -> pd.DataFrame:
    data = dict(np.load(filepath))
    df_data = pd.DataFrame(data['data'], columns=data['features'])
    df_target = pd.DataFrame(data['target'], columns=['target'])
    df = pd.concat([df_data, df_target], axis=1)
    return df

In [3]:
def get_ref(data: pd.DataFrame, metric:str, criteria: str):
    target = 1 if metric=='pd' else 0
    offline_approved = data[f'el_lh{criteria}'] == target
    cutbased_approved = data[triger_criterias[criteria]] == target
    res = {
        'total': offline_approved.sum(),
        'passed': (offline_approved & cutbased_approved).sum()
    }
    res[metric] = res['passed']/res['total']                        
    return res

In [4]:
et=4
eta=0
dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97'
loose_dataset = 'data17_13TeV.AllPeriods.sgn.probes_lhvloose_EGAM1.bkg.vprobes_vlhvloose_EGAM7.GRL_v97.25bins'
homepath = os.path.expanduser('~')
datapath = os.path.join(homepath, 'data', dataset)
loose_datapath = os.path.join(homepath, 'data', loose_dataset)
filepath = os.path.join(datapath, dataset + '_et{et}_eta{eta}.npz')
refpath = os.path.join(datapath, 'references')
reffilepath = os.path.join(refpath, dataset + '_et{et}_eta{eta}.ref.pic.gz')
loose_filepath = os.path.join(loose_datapath, loose_dataset + '_et{et}_eta{eta}.npz')

In [5]:
criterias = ['tight', 'medium', 'loose', 'vloose']
l2calo_trigers = ['T0HLTElectronT2CaloTight', 'T0HLTElectronT2CaloMedium', 'T0HLTElectronT2CaloLoose', 'T0HLTElectronT2CaloVLoose']
triger_criterias = {key: value for key, value in zip(criterias, l2calo_trigers)}
triger_criterias

{'tight': 'T0HLTElectronT2CaloTight',
 'medium': 'T0HLTElectronT2CaloMedium',
 'loose': 'T0HLTElectronT2CaloLoose',
 'vloose': 'T0HLTElectronT2CaloVLoose'}

In [6]:
ref = gaugi_load(reffilepath.format(et=et, eta=eta))
ref_total = ref['bkgRef']['medium_cutbased']['total'] + ref['sgnRef']['medium_cutbased']['total']
ref

{'class': 'Reference_v1',
 '__module': 'saphyra.readers.versions.Reference_v1',
 'etBinIdx': array(4),
 'etBins': array([1.5e+01, 2.0e+01, 3.0e+01, 4.0e+01, 5.0e+01, 1.0e+06]),
 '__version': 1,
 'bkgRef': OrderedDict([('tight_cutbased',
               {'total': 3769656,
                'passed': 763817.0,
                'reference': 'T0HLTElectronT2CaloTight'}),
              ('medium_cutbased',
               {'total': 3769656,
                'passed': 925500.0,
                'reference': 'T0HLTElectronT2CaloMedium'}),
              ('loose_cutbased',
               {'total': 3769656,
                'passed': 3444525.0,
                'reference': 'T0HLTElectronT2CaloLoose'}),
              ('vloose_cutbased',
               {'total': 3769656,
                'passed': 3481839.0,
                'reference': 'T0HLTElectronT2CaloVLoose'})]),
 'etaBinIdx': array(4),
 'etaBins': array([0.  , 0.8 , 1.37, 1.54, 2.37, 2.5 ]),
 'sgnRef': OrderedDict([('tight_cutbased',
               {

In [7]:
# Al the targets where 1 or 0 with floats, casted them to ints to avoid possible float comparision error
medium_data = load_as_df(filepath.format(et=et, eta=eta))
medium_total = medium_data.shape[0]
print(medium_data.shape)
print(medium_data['target'].dtype)
print(medium_data['el_lhtight'].dtype)
medium_data['el_lhtight'] = medium_data['el_lhtight'].astype('int16')
print(medium_data['el_lhmedium'].dtype)
medium_data['el_lhmedium'] = medium_data['el_lhmedium'].astype('int16')
print(medium_data['el_lhloose'].dtype)
medium_data['el_lhloose'] = medium_data['el_lhloose'].astype('int16')
print(medium_data['el_lhvloose'].dtype)
medium_data['el_lhvloose'] = medium_data['el_lhvloose'].astype('int16')
print(medium_data[triger_criterias['tight']].dtype)
medium_data[triger_criterias['tight']] = medium_data[triger_criterias['tight']].astype('int16')
print(medium_data[triger_criterias['medium']].dtype)
medium_data[triger_criterias['medium']] = medium_data[triger_criterias['medium']].astype('int16')
print(medium_data[triger_criterias['loose']].dtype)
medium_data[triger_criterias['loose']] = medium_data[triger_criterias['loose']].astype('int16')
print(medium_data[triger_criterias['vloose']].dtype)
medium_data[triger_criterias['vloose']] = medium_data[triger_criterias['vloose']].astype('int16')

(5469197, 155)
int16
float32
float32
float32
float32
float32
float32
float32
float32


In [17]:
for col in medium_data.columns:
    if not col.startswith('L2Calo_ring'):
        print(col)

avgmu
L2Calo_et
L2Calo_eta
L2Calo_phi
L2Calo_reta
L2Calo_ehad1
L2Calo_eratio
L2Calo_f1
L2Calo_f3
L2Calo_weta2
L2Calo_wstot
L2Calo_e2tsts1
L2Electron_hastrack
L2Electron_pt
L2Electron_eta
L2Electron_phi
L2Electron_caloEta
L2Electron_trkClusDeta
L2Electron_trkClusDphi
L2Electron_etOverPt
et
eta
phi
rhad1
rhad
f3
weta2
rphi
reta
wtots1
eratio
f1
hastrack
numberOfBLayerHits
numberOfPixelHits
numberOfTRTHits
d0
d0significance
eProbabilityHT
trans_TRT_PID
deltaEta1
deltaPhi2
deltaPhi2Rescaled
DeltaPOverP
deltaR
eeMass
el_lhtight
el_lhmedium
el_lhloose
el_lhvloose
T0HLTElectronT2CaloTight
T0HLTElectronT2CaloMedium
T0HLTElectronT2CaloLoose
T0HLTElectronT2CaloVLoose
target


In [8]:
((medium_data['target'] == medium_data['el_lhtight']).all(), 
(medium_data['target'] == medium_data['el_lhmedium']).all(), 
(medium_data['target'] == medium_data['el_lhloose']).all(), 
(medium_data['target'] == medium_data['el_lhvloose']).all())

(True, True, False, False)

In [9]:
((medium_data['target'] == medium_data[triger_criterias['tight']]).all(), 
 (medium_data['target'] == medium_data[triger_criterias['medium']]).all(), 
 (medium_data['target'] == medium_data[triger_criterias['loose']]).all(), 
 (medium_data['target'] == medium_data[triger_criterias['vloose']]).all())

(False, False, False, False)

In [10]:
medium_data_pds = {criteria: get_ref(medium_data, 'pd', criteria) for criteria in criterias}
medium_data_pds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


{'tight': {'total': 1699541, 'passed': 1693200, 'pd': 0.9962689926280096},
 'medium': {'total': 1699541, 'passed': 1694888, 'pd': 0.9972622019710028},
 'loose': {'total': 0, 'passed': 0, 'pd': nan},
 'vloose': {'total': 0, 'passed': 0, 'pd': nan}}

In [11]:
medium_data_fas = {criteria: get_ref(medium_data, 'fa', criteria) for criteria in criterias}
medium_data_fas

{'tight': {'total': 3769656, 'passed': 3005839, 'fa': 0.7973775325918333},
 'medium': {'total': 3769656, 'passed': 2844156, 'fa': 0.7544868815616067},
 'loose': {'total': 5469197, 'passed': 325630, 'fa': 0.05953890488859699},
 'vloose': {'total': 5469197, 'passed': 288226, 'fa': 0.05269987531990528}}

In [12]:
# Utilizando o filtro fornecido pelo Micael. Isso indica que o data medium já é o filtro aplicado sobre vloose (Acredito que seja sobre o vloose)
filtered_medium =  medium_data.loc[((medium_data['target'] == 1)&(medium_data['el_lhmedium'] == 1)) | ((medium_data['target'] != 1)&(medium_data['el_lhvloose'] !=1))]
filtered_medium_total = filtered_medium.shape[0]
filtered_medium.shape

(5469197, 155)

In [13]:
filtered_medium_data_pds = {criteria: get_ref(filtered_medium, 'pd', criteria) for criteria in criterias}
filtered_medium_data_pds

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


{'tight': {'total': 1699541, 'passed': 1693200, 'pd': 0.9962689926280096},
 'medium': {'total': 1699541, 'passed': 1694888, 'pd': 0.9972622019710028},
 'loose': {'total': 0, 'passed': 0, 'pd': nan},
 'vloose': {'total': 0, 'passed': 0, 'pd': nan}}

In [14]:
filtered_medium_data_fas = {criteria: get_ref(filtered_medium, 'fa', criteria) for criteria in criterias}
filtered_medium_data_fas

{'tight': {'total': 3769656, 'passed': 3005839, 'fa': 0.7973775325918333},
 'medium': {'total': 3769656, 'passed': 2844156, 'fa': 0.7544868815616067},
 'loose': {'total': 5469197, 'passed': 325630, 'fa': 0.05953890488859699},
 'vloose': {'total': 5469197, 'passed': 288226, 'fa': 0.05269987531990528}}

In [15]:
((filtered_medium['target'] == filtered_medium['el_lhtight']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhmedium']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhloose']).all(), 
(filtered_medium['target'] == filtered_medium['el_lhvloose']).all())

(True, True, False, False)

In [16]:
((filtered_medium['target'] == filtered_medium[triger_criterias['tight']]).all(), 
 (filtered_medium['target'] == filtered_medium[triger_criterias['medium']]).all(), 
 (filtered_medium['target'] == filtered_medium[triger_criterias['loose']]).all(), 
 (filtered_medium['target'] == filtered_medium[triger_criterias['vloose']]).all())

(False, False, False, False)

In [18]:
vloose_data = kepler_load(loose_filepath.format(et=et, eta=eta))
vloose_total = vloose_data.shape[0]
vloose_data.shape

(5599840, 216)

In [19]:
for col in vloose_data.columns:
    if not col.startswith('trig_L2_cl_ring'):
        print(col)

RunNumber
avgmu
trig_L2_cl_et
trig_L2_cl_eta
trig_L2_cl_phi
trig_L2_cl_reta
trig_L2_cl_ehad1
trig_L2_cl_eratio
trig_L2_cl_f1
trig_L2_cl_f3
trig_L2_cl_weta2
trig_L2_cl_wstot
trig_L2_cl_e2tsts1
trig_L2_el_hastrack
trig_L2_el_pt
trig_L2_el_eta
trig_L2_el_phi
trig_L2_el_caloEta
trig_L2_el_trkClusDeta
trig_L2_el_trkClusDphi
trig_L2_el_etOverPt
trig_EF_cl_hascluster
trig_EF_cl_et
trig_EF_cl_eta
trig_EF_cl_etaBE2
trig_EF_cl_phi
trig_EF_el_hascand
trig_EF_el_et
trig_EF_el_eta
trig_EF_el_etaBE2
trig_EF_el_phi
trig_EF_el_rhad1
trig_EF_el_rhad
trig_EF_el_f3
trig_EF_el_weta2
trig_EF_el_rphi
trig_EF_el_reta
trig_EF_el_wtots1
trig_EF_el_eratio
trig_EF_el_f1
trig_EF_el_hastrack
trig_EF_el_deltaEta1
trig_EF_el_deltaPhi2
trig_EF_el_deltaPhi2Rescaled
trig_EF_el_lhtight
trig_EF_el_lhmedium
trig_EF_el_lhloose
trig_EF_el_lhvloose
el_et
el_eta
el_etaBE2
el_phi
el_rhad1
el_rhad
el_f3
el_weta2
el_rphi
el_reta
el_wtots1
el_eratio
el_f1
el_hastrack
el_numberOfBLayerHits
el_numberOfPixelHits
el_numberOfTRTHits
e

In [20]:
vloose_data['trig_L2_cl_lhvloose_et0to12'].head()

0    True
1    True
2    True
3    True
4    True
Name: trig_L2_cl_lhvloose_et0to12, dtype: bool

In [21]:
vloose_data['trig_L2_cl_lhvloose_et0to12'].unique()

array([ True, False])

In [22]:
vloose_data['trig_L2_cl_lhvloose_et12to20'].head()

0    True
1    True
2    True
3    True
4    True
Name: trig_L2_cl_lhvloose_et12to20, dtype: bool

In [23]:
vloose_data['trig_L2_cl_lhvloose_et12to20'].unique()

array([ True, False])

In [24]:
is_equal = (vloose_data['trig_L2_cl_lhvloose_et0to12'] == vloose_data['trig_L2_cl_lhvloose_et12to20'])
is_equal.all(), is_equal.any()

(False, True)

In [25]:
((vloose_data['target'] == vloose_data['el_lhtight']).all(), 
(vloose_data['target'] == vloose_data['el_lhmedium']).all(), 
(vloose_data['target'] == vloose_data['el_lhloose']).all(), 
(vloose_data['target'] == vloose_data['el_lhvloose']).all())

(False, False, False, True)

In [26]:
filtered_vloose = vloose_data.loc[((vloose_data.target == 1)&(vloose_data.el_lhmedium == 1)) | ((vloose_data.target != 1)&(vloose_data.el_lhvloose !=1))]
filtered_vloose_total = filtered_vloose.shape[0]
filtered_vloose.shape

(5469055, 216)

In [27]:
((filtered_vloose['target'] == filtered_vloose['el_lhtight']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhmedium']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhloose']).all(), 
(filtered_vloose['target'] == filtered_vloose['el_lhvloose']).all())

(False, True, True, True)

In [28]:
ref_total, medium_total, filtered_medium_total, vloose_total, filtered_vloose_total

(5420314, 5469197, 5469197, 5599840, 5469055)